### Importing Libraries

In [32]:
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt

### Cloud Detection

1. INSAT-3D TIR captures preprocessed using gdal and rasterio packages.
2. Apply KMeans clustering with k set to 4.
3. Remove background considering darkest cluster to be clear sky.
4. Return Centre Of Mass for every cloud cluster.

In [43]:
def cloud_detection(timestamp):
    img = cv2.imread('./images/' + timestamp + '.png')
    Z = img.reshape((-1,3))

    # convert to np.float32
    Z = np.float32(Z)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 4
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))
    res2bw = cv2.cvtColor(res2, cv2.COLOR_BGR2GRAY)
    colors = np.unique(res2bw)

    # Removing background, since the darkest color is sky
    colors = colors[1:]
    # print(colors)

    com = {}
    for i in range(0, len(colors)):
        filter_mask = (res2bw == colors[i]).astype(int)
        filter_mask = cv2.convertScaleAbs(filter_mask, alpha=(255.0))
        co_ordinates = np.where(filter_mask == 0)
        x = co_ordinates[0]
        y = co_ordinates[1]
        mean_x = np.mean(x)
        mean_y = np.mean(y)
        com[i] = [round(mean_x, 2), round(mean_y, 2)]
    
    return com

### Mean Path Adjustment

1. Let previous CoM positions be: t - 1, t - 2
2. Let at be the actual CoM position at t
3. Predicted mean CoM at t = mean(t - 1, t - 2)
4. Next prediction for t+1 = mean(at, t) + (at-t)

Adopts itself to abrupt changes in cloud positions by:
- Using actual and predicted values for next prediction 
- Correcting its prediction by adding the errors

In [44]:
def mean_path_adjustment(com1, com2, com3, com4): 
    predicted = {}
    error = {}
    for cloud_index in range(0, len(com1)):
        x1 = com1[cloud_index][0]
        y1 = com1[cloud_index][1]

        x2 = com2[cloud_index][0]
        y2 = com2[cloud_index][1]

        x3 = com3[cloud_index][0]
        y3 = com3[cloud_index][1]

        x4 = com4[cloud_index][0]
        y4 = com4[cloud_index][1]

        x = (x1 + x2) / 2
        y = (y1 + y2) / 2

        predicted_x4 = ((x + x3) / 2) + (x3 - x)
        predicted_y4 = ((y + y3) / 2) + (y3 - y)

        predicted[cloud_index] = [predicted_x4, predicted_y4]
        error[cloud_index] = round(math.sqrt(pow(abs(predicted_x4 - x4), 2) + pow(abs(predicted_y4 - y4), 2)), 2)

    '''print("Actual COM at t + 1: ", com4)
    print("Predicted COM at t + 1: ", predicted)
    print("Error: ", error)'''
    
    return com4, predicted, error

### Testing MPA

In [38]:
com_dict = {}
error = {}
predicted = {}
for i in range(2, 41):
    if i not in com_dict:
        com_dict[i] = cloud_detection(str(i))
    
    if i+1 not in com_dict:
        com_dict[i+1] = cloud_detection(str(i+1))
    
    if i+2 not in com_dict:
        com_dict[i+2] = cloud_detection(str(i+2))
    
    if i+3 not in com_dict:
        com_dict[i+3] = cloud_detection(str(i+3))
    
    actual, predicted[i], error[i] = mean_path_adjustment(com_dict[i], com_dict[i+1], com_dict[i+2], com_dict[i+3])

### Error Calculations

In [41]:
import pandas as pd
df = pd.DataFrame(error)
df

,2,3,4,5,6,7,8,9,10,11,...,31,32,33,34,35,36,37,38,39,40
0,36.39,43.35,53.55,24.68,7.60,10.22,3.32,5.23,4.96,5.01,...,2.54,0.95,8.65,9.57,9.16,2.67,1.39,1.80,3.34,2.79
1,55.86,16.65,42.56,17.05,3.10,19.18,8.32,2.77,1.44,0.40,...,0.73,1.59,10.48,9.05,7.94,5.20,1.57,1.78,3.35,4.30
2,0.94,0.87,1.61,0.48,0.29,1.70,1.80,0.86,0.32,0.71,...,0.36,0.81,0.50,2.52,1.30,1.40,0.64,0.61,0.32,0.61


In [45]:
df.mean(axis=1)

0    11.842051
1    11.019487
2     1.248718
dtype: float64